# Notebook `02_model_regression.ipynb`

---

## 📈 Entraînement des modèles de régression

Ce notebook a pour objectif de construire et évaluer différents **modèles de régression** pour prédire le **prix d’un service Fiverr** à partir de ses caractéristiques (texte, fiabilité, etc.).

Nous comparons ici plusieurs algorithmes supervisés, avec ou sans transformation logarithmique, pour identifier le modèle le plus robuste.

## 🎯 Objectifs

- Charger les features et les cibles préparées (`X_scaled`, `y_reg`)
- Tester plusieurs modèles de régression (linéaire, arbres, boosting, etc.)
- Évaluer les performances avec des métriques comme **MAE**, **RMSE** et **R²**
- Sélectionner le meilleur modèle pour la prédiction du **prix réel**
- Sauvegarder le modèle retenu pour les étapes de prédiction et de mise en production

## ✅ Compétences mobilisées

- **Bloc 3 — C1** : Comparer les performances de plusieurs algorithmes de régression pour choisir le plus adapté à la problématique.
- **Bloc 3 — C2** : Adapter les données à la forme attendue par les modèles (notamment via `StandardScaler` et transformation `log`).
- **Bloc 3 — C3** : Entraîner un modèle de régression en optimisant ses performances selon des indicateurs clairement définis (MAE, RMSE, R²).

*Ce notebook permet de poser les fondations du moteur de prédiction de prix utilisé dans l'application finale.*

---

## 🧭 Sommaire

1. [Importation des bibliothèques](#-1-importation-des-bibliothèques)
2. [Chargement des données transformées](#-2-chargement-des-données-transformées)
3. [Construction des variables explicatives (`X`) et cibles (`y`)](#-3-construction-des-variables-explicatives-x-et-cibles-y)
4. [Séparation des données et définition des modèles](#-4-séparation-des-données-et-définition-des-modèles)
5. [Entraînement, évaluation et sauvegarde des modèles de régression](#-5-entraînement-évaluation-et-sauvegarde-des-modèles-de-régression)
6. [Sauvegarde du préprocesseur et des colonnes de features](#-6-sauvegarde-du-préprocesseur-et-des-colonnes-de-features)
7. [Résultats comparatifs des modèles de régression](#-7-résultats-comparatifs-des-modèles-de-régression)
8. [Sélection du meilleur modèle de régression](#-8-sélection-du-meilleur-modèle-de-régression)

---

## 📘 1. Importation des bibliothèques

### ❓ 1.1. Pourquoi cette étape ?

Avant d’entamer le processus de modélisation, il est indispensable d’importer toutes les bibliothèques nécessaires au traitement des données, à l’entraînement des modèles et à leur évaluation.

Les modules importés ici permettent de :

- **Charger et manipuler** les données (`pandas`, `numpy`, `os`, `joblib`)
- **Préparer les jeux de données** (`train_test_split`, `StandardScaler`)
- **Entraîner plusieurs types de modèles de régression** :
  - `LinearRegression`, `Ridge`
  - `DecisionTreeRegressor`, `RandomForestRegressor`
  - `GradientBoostingRegressor`, `XGBRegressor`
  - `KNeighborsRegressor`
- **Évaluer les performances** via des métriques (`MAE`, `RMSE`, `R²`)
- **Générer des embeddings textuels** à partir de la description (`SentenceTransformer`)

---

### 🐍 1.2. Script d’importation des bibliothèques

In [ ]:
# Importation des bibliothèques nécessaires à la modélisation

# Manipulation de données
import pandas as pd                                              # Manipulation de DataFrames
import numpy as np                                               # Fonctions mathématiques avancées
import os                                                        # Interaction avec le système de fichiers
import joblib                                                    # Sauvegarde et chargement de modèles

# Préparation des données
from sklearn.model_selection import train_test_split             # Découpe en train/test
from sklearn.preprocessing import StandardScaler                 # Normalisation des données numériques

# Modèles de régression standards
from sklearn.linear_model import LinearRegression, Ridge         # Régressions linéaires (classique et régularisée)
from sklearn.ensemble import RandomForestRegressor               # Forêts aléatoires
from sklearn.ensemble import GradientBoostingRegressor           # Boosting
from sklearn.tree import DecisionTreeRegressor                   # Arbre de régression simple
from sklearn.neighbors import KNeighborsRegressor                # Régression par les k plus proches voisins
from xgboost import XGBRegressor                                 # Régression boostée performante (XGBoost)

# Évaluation des performances
from sklearn.metrics import mean_absolute_error                  # - MAE : erreur absolue moyenne
from sklearn.metrics import  mean_squared_error                  # - MSE : erreur quadratique moyenne
from sklearn.metrics import r2_score                             # - R² : coefficient de détermination (qualité d'ajustement)

# NLP - Embeddings de texte
from sentence_transformers import SentenceTransformer            # Génération de vecteurs à partir de texte

---

## 📦 2. Chargement des données transformées

### ❓ 2.1. Pourquoi cette étape maintenant ?

Après le prétraitement complet du jeu de données brut, nous avons sauvegardé une version transformée et enrichie (`fiverr_cleaned_transformed.csv`).  
Cette étape consiste à **recharger ce fichier préparé** pour démarrer la phase de modélisation.

Ce fichier contient :
- Des **colonnes nettoyées et prêtes à l’emploi** : `Description`, `Niveau`, `Prix`, `Fiabilite`, etc.
- Les **valeurs manquantes imputées**
- Les descriptions textuelles **nettoyées des stopwords** et formules types
- Des formats unifiés (`float`, `str`, etc.)

### 🎯 2.2. Résultat attendu

- Les données sont chargées dans un objet `DataFrame` nommé `df`.
- Elles sont prêtes à être utilisées pour la phase de modélisation (régression et/ou classification).

---

### 🐍 2.3. Script de chargement des données transformées

In [ ]:
# Chargement du jeu de données transformé

# Lecture du fichier CSV contenant les données nettoyées et enrichies
# - Le fichier 'fiverr_cleaned_transformed.csv' est issu des étapes précédentes de préprocessing.
# - Il contient déjà les colonnes prêtes à être utilisées pour l'entraînement des modèles (ex. : Description nettoyée, Fiabilité, Niveau, Prix, etc.)

df = pd.read_csv("../data/fiverr_cleaned_dl_notebook.csv")

---

## 🧠 3. Construction des variables explicatives (`X`) et cibles (`y`)

### ❓ 3.1. Pourquoi cette étape maintenant ?

Nous allons maintenant préparer les **données d’entrée du modèle** (`X`) et la **variable à prédire** (`y`), à partir du fichier transformé.  
Les colonnes choisies sont prêtes à l’emploi grâce aux étapes précédentes (nettoyage, vectorisation, standardisation).

### 🔧 3.2. Détails des transformations effectuées

| Étape | Description |
|-------|-------------|
| 🧩 Embedding | Les titres de service (`Description`) sont transformés en **vecteurs numériques de 384 dimensions** via un modèle pré-entraîné (`all-MiniLM-L6-v2`). |
| ⚖️ Normalisation | La variable `Fiabilite` est **standardisée** à l’aide d’un `StandardScaler`. Cela améliore la performance de nombreux modèles. |
| 🧷 Fusion | Les embeddings et la fiabilité standardisée sont **fusionnés horizontalement** dans une matrice `X` utilisée pour l'entraînement. |
| 🎯 Cibles | Deux cibles sont définies : `y_log` (log du prix pour l'entraînement) et `y_real` (prix réel pour l’évaluation des performances). |

> 💡 Remarque : le one-hot encoding de `Niveau` peut être réintégré ultérieurement si l'on souhaite inclure cette variable dans les features.

---

### 🐍 3.3. Script de construction des variables explicatives

In [23]:
# Génération des embeddings à partir des descriptions textuelles

# Modèle pré-entraîné pour transformer le texte en vecteurs numériques (384 dimensions)
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Transformation de chaque description textuelle en vecteur dense
# - On s'assure que la colonne Description est bien convertie en chaîne de caractères
# - Le paramètre show_progress_bar=True affiche une barre de progression utile en cas de grand volume
embeddings = embedding_model.encode(df["Description"].astype(str).tolist(), show_progress_bar=True)

# Conversion en DataFrame avec noms explicites pour chaque dimension
embed_df = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(384)])

# Encodage one-hot du niveau (commenté ici si non utilisé)
# - Utile si l'on souhaite intégrer le niveau du vendeur dans le modèle
# niveau_encoded = pd.get_dummies(df["Niveau"], prefix="Niveau")

# Standardisation de la variable Fiabilité

# - Le StandardScaler transforme la variable pour qu’elle ait une moyenne de 0 et un écart-type de 1
# - Cela facilite la convergence des algorithmes sensibles à l’échelle (ex. : régression, KNN...)
scaler = StandardScaler()
fiabilite_scaled = scaler.fit_transform(df[["Fiabilite"]])

# Conversion en DataFrame avec nom de colonne conservé
fiabilite_df = pd.DataFrame(fiabilite_scaled, columns=["Fiabilite"])

# Fusion des différentes sources de données

# - On concatène horizontalement les embeddings textuels et la fiabilité normalisée
# - Le DataFrame final `X` est l’ensemble des features utilisées pour l’entraînement
X = pd.concat([embed_df, fiabilite_df], axis=1)

# Cibles à prédire
# - y_log : version logarithmique du prix (pour modèle)
# - y_real : prix réel (pour interprétation ou évaluation)
y_log = df["Prix_log"]
y_real = df["Prix"]

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

---

## 🧪 4. Séparation des données et définition des modèles

### ❓ 4.1. Pourquoi cette étape maintenant ?

Avant de tester nos modèles, il est indispensable de :
1. **Diviser le jeu de données** en un ensemble d'entraînement (80 %) et un ensemble de test (20 %) ;
2. **Définir une sélection de modèles** à comparer objectivement sur les mêmes données.

### 🔄 4.2. Détails de la division train/test

| Variable        | Rôle                                      |
|----------------|-------------------------------------------|
| `X_train`       | Données d'entraînement (features)         |
| `X_test`        | Données de test (features)                |
| `y_train`       | Prix log transformé — à prédire (train)   |
| `y_test`        | Prix log transformé — à prédire (test)    |
| `y_real_train`  | Prix réel pour comparaison éventuelle     |
| `y_real_test`   | Prix réel pour l’évaluation des erreurs   |

### 🤖 4.3. Modèles sélectionnés pour la régression

| Modèle               | Description rapide                           |
|----------------------|----------------------------------------------|
| `LinearRegression`   | Régression linéaire classique                |
| `Ridge`              | Régression linéaire avec régularisation L2   |
| `RandomForest`       | Agrégation d’arbres décisionnels (bagging)   |
| `GradientBoosting`   | Entraînement séquentiel d’arbres faibles     |
| `XGBoost`            | Boosting optimisé très performant            |
| `DecisionTree`       | Arbre de décision unique                     |
| `KNN Regressor`      | Moyenne des k plus proches voisins           |

> ⚠️ Tous les modèles seront entraînés sur les **mêmes données** et évalués selon des métriques identiques pour un comparatif juste.

---

### 🐍 4.4. Script de séparation des données et définition des modèles

In [ ]:
# Séparation des données en ensembles d'entraînement et de test

# X         : matrice des caractéristiques (embeddings + fiabilité)
# y_log     : prix transformé en échelle logarithmique (cible d'entraînement)
# y_real    : prix réel (utilisé uniquement pour l'évaluation, pas pour l'entraînement)

# Paramètres :
# - test_size=0.2        : 20 % des données seront utilisées pour les tests
# - random_state=42      : graine aléatoire pour garantir la reproductibilité

X_train, X_test, y_train, y_test, y_real_train, y_real_test = train_test_split(
    X, y_log, y_real, test_size=0.2, random_state=42
)

# Définition des modèles de régression à comparer

# Chaque modèle est instancié avec des paramètres de base cohérents
models = {
    "Linear Regression": LinearRegression(),                                            # - LinearRegression : modèle linéaire de base
    "Ridge": Ridge(alpha=1.0),                                                          # - Ridge : régression linéaire avec régularisation L2      
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),          # - Random Forest    : ensemble d’arbres (100 arbres, aléatoire fixé)
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),  # - Gradient Boosting: boosting d’arbres (100 itérations, aléatoire fixé)
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42, verbosity=0),            # - XGBoost          : gradient boosting très performant (100 arbres, verbosité coupée)    
    "Decision Tree": DecisionTreeRegressor(random_state=42),                            # - Decision Tree    : arbre unique, simple à interpréter
    "KNN Regressor": KNeighborsRegressor(n_neighbors=5)                                 # - KNN              : régression par les k plus proches voisins (ici k=5)
}

---

## 🧠 5. Entraînement, évaluation et sauvegarde des modèles de régression

### 🎯 5.1. Objectif

Comparer les performances de plusieurs modèles de régression sur la prédiction du **prix réel**, à partir de la cible log-transformée (`Prix_log`).  
Nous allons :
- entraîner chaque modèle sur les mêmes données (`X_train`, `y_train`),
- prédire sur les mêmes données de test (`X_test`),
- évaluer les performances selon 3 métriques principales.

### 📏 5.2. Métriques utilisées

| Métrique | Signification                             |
|----------|-------------------------------------------|
| MAE      | Erreur absolue moyenne                    |
| RMSE     | Erreur quadratique moyenne                |
| R²       | Coefficient de détermination              |

> 🔁 Tous les résultats sont **calculés à partir des prix réels** (après transformation inverse du log).

### 💾 5.3. Sauvegarde automatique

Chaque modèle est enregistré automatiquement dans le dossier :


### 🥇 5.4. Sélection du meilleur modèle

Le **modèle ayant le plus petit RMSE** est conservé comme meilleur modèle (`best_model`), et son nom est stocké pour une utilisation future.

---

### 🐍 5.5. Script de sélection du meilleur modèle


In [ ]:
# Entraînement et évaluation de tous les modèles de régression

results = []                          # Liste pour stocker les scores de chaque modèle
best_model = None                    # Pour conserver le meilleur modèle trouvé
best_rmse = float('inf')             # Initialisation du plus petit RMSE à l’infini

# Boucle sur chaque modèle du dictionnaire
for name, model in models.items():
    # Entraînement du modèle sur l'ensemble d'entraînement
    model.fit(X_train, y_train)

    # Prédiction sur l’ensemble de test (prix en échelle log)
    y_pred_log = model.predict(X_test)

    # Transformation inverse pour retrouver les prix réels
    y_pred = np.expm1(y_pred_log)

    # Calcul des métriques d’évaluation
    mae = mean_absolute_error(y_real_test, y_pred)  # Erreur absolue moyenne
    rmse = np.sqrt(mean_squared_error(y_real_test, y_pred))  # Erreur quadratique moyenne
    r2 = r2_score(y_real_test, y_pred)  # Coefficient de détermination

    # Stockage des résultats arrondis dans une liste de dictionnaires
    results.append({
        "Modèle": name,
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "R²": round(r2, 4)
    })

    # Sauvegarde du modèle entraîné dans le dossier approprié
    model_filename = f"{name.replace(' ', '_').lower()}_notebook.pkl"
    joblib.dump(model, f"../models/regression/{model_filename}")

    # Suivi du meilleur modèle (basé sur le plus petit RMSE)
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_name = name


---

## 💾 6. Sauvegarde du préprocesseur et des colonnes de features

### 🎯 6.1. Objectif

Pour garantir que le pipeline de prédiction future soit **reproductible et cohérent**, il est indispensable de sauvegarder :
- le **scaler** utilisé pour la standardisation de la colonne `Fiabilite` (ici : `StandardScaler`),
- la **liste exacte des colonnes** utilisées comme features (noms et ordre).

Cela évite les erreurs de transformation ou de dimensions lors de l'inférence en production ou dans l’application Gradio.

### 🧱 6.2. Éléments sauvegardés

| Élément         | Chemin de sauvegarde                  | Description |
|-----------------|----------------------------------------|-------------|
| `scaler`        | `models/regression/scaler.pkl`        | Objet `StandardScaler` entraîné sur la fiabilité |
| `columns_used`  | `models/columns_used.pkl`             | Liste ordonnée des noms de colonnes de `X` |

---

### 🐍 6.3. Script de sauvegarde

In [26]:
# Sauvegarde du scaler et des colonnes utilisées

# Le modèle sélectionné (best_model) a déjà été sauvegardé précédemment dans la boucle.
# Ici, nous sauvegardons le préprocesseur utilisé (StandardScaler) et la liste des colonnes du jeu de données final.

# Sauvegarde du scaler utilisé pour la standardisation de la variable 'Fiabilite'
# Cela permettra de reproduire exactement la même transformation à l'inférence
joblib.dump(scaler, "../models/regression/scaler_notebook.pkl")

# Sauvegarde de la liste des colonnes utilisées dans X (ordre et noms)
# Utile pour reconstituer la même matrice de features à la prédiction
joblib.dump(X.columns.tolist(), "../models/columns_used_notebook.pkl")

['../models/columns_used_notebook.pkl']

---

## 🏁 7. Résultats comparatifs des modèles de régression

### 📊 7.1. Tableau de synthèse des performances

Une fois les modèles entraînés, nous comparons leurs performances sur l’ensemble de test à l’aide des métriques suivantes :

| Modèle              | MAE  | RMSE | R²    |
|---------------------|------|------|-------|
| ... (ex. Ridge, RF) | ...  | ...  | ...   |

Le tableau est trié selon la **valeur croissante du RMSE**, afin de visualiser directement les modèles les plus performants.

> ℹ️ Toutes les métriques sont calculées **sur les prix réels**, après transformation inverse du logarithme.

### 🥇 7.2. Meilleur modèle sélectionné

Le **modèle ayant obtenu le plus faible RMSE** est automatiquement sélectionné comme modèle final (`best_model`).  
Son nom est affiché à la fin du tableau pour confirmation.

---

### 🐍 7.3. Script d’affichage des performances

In [27]:
# Affichage des résultats comparatifs des modèles de régression

# Transformation de la liste des résultats en DataFrame et tri par RMSE (croissant)
df_results = pd.DataFrame(results).sort_values("RMSE")

# Affichage formaté des résultats en tableau Markdown
print("Résultats comparatifs (évalués sur les vrais prix) :\n")
print(df_results.to_markdown(index=False))  # affichage lisible dans les notebooks / consoles

# 🥇 Rappel du meilleur modèle
print(f"\nMeilleur modèle : {best_name} (RMSE = {round(best_rmse, 2)})")

📊 Résultats comparatifs (évalués sur les vrais prix) :

| Modèle            |   MAE |   RMSE |      R² |
|:------------------|------:|-------:|--------:|
| Gradient Boosting |  3.21 |   4.9  |  0.2566 |
| XGBoost           |  3.33 |   5    |  0.2274 |
| Random Forest     |  3.32 |   5.03 |  0.2173 |
| Ridge             |  3.82 |   5.47 |  0.0748 |
| KNN Regressor     |  3.99 |   5.8  | -0.0407 |
| Decision Tree     |  4.43 |   7.14 | -0.577  |
| Linear Regression |  5.86 |   9.9  | -2.0358 |

✅ Meilleur modèle : Gradient Boosting (RMSE = 4.9)


## 🏆 8. Sélection du meilleur modèle de régression

Après avoir entraîné et comparé **7 modèles de régression**, leurs performances ont été évaluées selon trois métriques essentielles :  
- **MAE** : Erreur absolue moyenne (plus c’est bas, mieux c’est)  
- **RMSE** : Racine de l'erreur quadratique moyenne (plus c’est bas, mieux c’est)  
- **R²** : Coefficient de détermination (plus c’est proche de 1, mieux c’est)

---

### 📊 8.1. Résultats comparatifs (évalués sur les vrais prix)

| Modèle            |   MAE |   RMSE |    R²    |
|-------------------|-------|--------|----------|
| **Gradient Boosting** |  3.21 |   4.90 | **0.2566** |
| XGBoost           |  3.33 |   5.00 | 0.2274   |
| Random Forest     |  3.32 |   5.03 | 0.2173   |
| Ridge             |  3.82 |   5.47 | 0.0748   |
| KNN Regressor     |  3.99 |   5.80 | -0.0407  |
| Decision Tree     |  4.43 |   7.14 | -0.5770  |
| Linear Regression |  5.86 |   9.90 | -2.0353  |

✅ **Meilleur modèle : Gradient Boosting (RMSE = 4.90)**

---

ℹ️ **Remarque sur le choix du modèle**

 Le modèle `Gradient Boosting` s’est imposé comme le plus performant sur les données de test, avec :
 - la **plus faible erreur quadratique moyenne (RMSE)**,
 - un **coefficient R² positif**, montrant une bonne capacité de généralisation.

Ce modèle sera donc utilisé pour la suite de l’analyse et dans l’application finale.